# Everything's Linear with the Right Kernel Function
_Exploring high-dimensional space with Support Vector Machines_

#### Data Science 420 BB
#### University of Washington Professional & Continuing Education
#### Homework 5: Support Vector Machines
#### Leo Salemann, 5/22/18

# Question 1

**Explain the differences between traditional two-class Support Vector Machines (SMVs) versus two-class Locally-Deep Support Vector Machines (LD-SVMs). Provide examples of when you will prefer to use one over the other.**

## Two-Class SVMs in General

Both types of Support Vector Machines (SVMs, LD-SVMs) are designed to solve binary classification problems by finding a hyperplane that separates the input data into two classes. If the data is two-dimensional (only two features per observation), the hyperplane becomes a line; for three dimensional data it beocmes a regular 2D plane, and for N-dimensions (data with N features per observation) it beocmes an N-1 dimensional hyperplane. The key point here is that SVMs are _linear_ classifiers - meaning that the line mentioned for the 2D case the hyperplane is an actual straight line not a curve; for the 3D case it's a flat 2D plane not a wavy surface; and for higher dimensions the hyperplane is still "flat" in whatever sense that may mean in N-1 space. In cases where the data doesn't lend it self to classificaiton by a "flat" separate (i.e. most of the time) a _Kernel Function_ is employed to project the data into a higher dimensional space in which a "flat" separator works. 

Consider a random collection og pixels sampled from an image of a fried egg. We have [x, y, color] and want to clasify yolk (round, in the center)  vs. egg white (backgroud, everythere else). No straight line will separate our yolk from white, but if we project our dataset into 3D space, the yolk becomes a "mountain" or "hole" and a flat plane will "slice it" from the egg white quite neatly.  

SVM's offer a variety of kernel functions to (Polynomial, Gaussian, Sigmoid, RBD, ...) to acommplish feats like this in multi-dimensional space.  The tricky part is finding the right one, and putting in the time to test & tune it.  Got a big data set, hope you have patientce. These kernels can be slow. 

## What's so special about LD-SVMs?

[Locally-Deep Support Vector Machines (LD-SVMs)](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/two-class-locally-deep-support-vector-machine) are essentially SVM's driven by [Local Deep Kernel Learning (LKDL)](http://manikvarma.org/code/LDKL/download.html), a kernel technique that addresses the projection needs descibed in the previous seciton, in a more efficent manner. LD-SVMs trade a bit of classification accuracy (usually less than a percentage point, occasionaly up to five) in favor of mulitple orders of magnitude peformance speedups. 

## How to Choose, which one to use?

Those who worship speed (or shiny new things in general) may be tempted to go all-in with LD-SVMs right of the bat, but a few preliminary considerations can guide you to a wiser choice.

**Do you have infrastructure constraints?** If you want to get started with readymade LD-SVMs, Azure ML Stodio is pretty much the only game in town (as of May 2018). You'll have a hard time finding R packages or Python libaries to help you, though you might be able to build a wrapper over [Varma et. all's LKDL Kerel implemented in C++](http://manikvarma.org/code/LDKL/LDKL.zip). Even if you have complete freedom with respect to choice of language, you may still have Cloud vs. On-Premesis constraints. Understand your client, data sets, and terms of data use before building pipelines you're not allowed to use. Keep up with cloud security data proteciton capabilities (multiple providers provide government-level proteciton), or look into [how to run the Azure stack On-Premises](https://azure.microsoft.com/en-us/resources/videos/microsoft-azure-stack-azure-services-on-premises/) 

**How big is your data?** LD-SVM sings when up against tens or even hundreds of thousands observaations with tens or hundreds of dimensions. 

**Can you tolerate a loss of accuracy?** LD-SVM will usually cost you some accuracy in favor of speed. [the characteristics of your data [Jain et al fig. 2](http://manikvarma.org/code/LDKL/download.html) as well as your tuning pareamters can have en effeect. If every once of accruacy counts, consider staying with conventional SVM.

**Have you tried the easy way first?** Most clients appreciate partial progress now rather than a complete solution never. Unless your data is aboslutely massive, you can probably try a few standard kernel functions with conventional SVM, in the language and on the infrastructure of your choice, to see where you stand. Step up to LD-SVM if conventional SVM fails you.

Finallly, [keep an eye on the forums](https://social.msdn.microsoft.com/Forums/azure/en-US/b809782e-2169-4aa0-b512-18e6542e6600/when-would-you-use-two-class-locally-deep-svm-vs-normal-svm?forum=MachineLearning) to see if the situation changes. 

# Question 2
**Use the veh-prime dataset to classify cars vs. non-cars with SVMs.  Try 10 fold cross-validation, tune parameters and observe results.**

In [1]:
options(repr.plot.width=7, repr.plot.height=6)

## Data Prep

### Load the data, have a look

In [2]:
mainDF = read.csv("veh-prime.csv")
head(mainDF, 3)
dim(mainDF)

f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,⋯,f27,f28,f29,f30,f31,f32,f33,f34,f35,CLASS
0.063,0.160,0.509,-0.967,0.058,0.000,0.874,0.271,1.307,-0.011,⋯,-0.924,-0.077,0.108,-0.003,0.381,-0.314,0.929,0.184,-0.001,noncar
-0.037,-0.325,-0.626,-0.029,0.121,-0.409,-0.002,-0.835,-0.595,-0.253,⋯,0.270,0.533,0.152,-0.978,0.157,0.011,-0.254,0.453,-0.621,noncar
0.000,1.253,0.833,-0.970,1.516,0.014,-0.378,1.197,0.546,-0.402,⋯,-0.408,1.550,0.010,-0.652,-0.403,-0.151,0.000,0.049,-0.113,car


[1] 846  37

800-ish observations with 40-sh observations.

### Check for NA's and overall value spread

In [3]:
summary(mainDF)

       f0                 f1                   f2            
 Min.   :-0.99900   Min.   :-2.5110000   Min.   :-1.9230000  
 1st Qu.:-0.12075   1st Qu.:-0.8110000   1st Qu.:-0.7880000  
 Median : 0.00000   Median :-0.0820000   Median :-0.1400000  
 Mean   : 0.01666   Mean   : 0.0000154   Mean   :-0.0000437  
 3rd Qu.: 0.15850   3rd Qu.: 0.7680000   3rd Qu.: 0.6710000  
 Max.   : 0.99700   Max.   : 3.0750000   Max.   : 2.2920000  
       f3                  f4                   f5          
 Min.   :-0.992000   Min.   :-2.6690000   Min.   :-0.98700  
 1st Qu.:-0.126750   1st Qu.:-0.7660000   1st Qu.:-0.13900  
 Median : 0.000000   Median :-0.1320000   Median : 0.00000  
 Mean   : 0.001991   Mean   : 0.0000674   Mean   :-0.01957  
 3rd Qu.: 0.121000   3rd Qu.: 1.0090000   3rd Qu.: 0.08450  
 Max.   : 0.997000   Max.   : 1.8970000   Max.   : 0.99500  
       f6                  f7                  f8                 f9           
 Min.   :-0.999000   Min.   :-1.940000   Min.   :-1.86300  

Everything but the target variable is a numeric, no NaN's, most features range beteen +/- 1 (though not all)

### Replace string-based target variable with a numeric

In [4]:
mainDF$TARGET <- ifelse(mainDF$CLASS =='car',1,-1)
mainDF$CLASS <- NULL
summary(mainDF)

       f0                 f1                   f2            
 Min.   :-0.99900   Min.   :-2.5110000   Min.   :-1.9230000  
 1st Qu.:-0.12075   1st Qu.:-0.8110000   1st Qu.:-0.7880000  
 Median : 0.00000   Median :-0.0820000   Median :-0.1400000  
 Mean   : 0.01666   Mean   : 0.0000154   Mean   :-0.0000437  
 3rd Qu.: 0.15850   3rd Qu.: 0.7680000   3rd Qu.: 0.6710000  
 Max.   : 0.99700   Max.   : 3.0750000   Max.   : 2.2920000  
       f3                  f4                   f5          
 Min.   :-0.992000   Min.   :-2.6690000   Min.   :-0.98700  
 1st Qu.:-0.126750   1st Qu.:-0.7660000   1st Qu.:-0.13900  
 Median : 0.000000   Median :-0.1320000   Median : 0.00000  
 Mean   : 0.001991   Mean   : 0.0000674   Mean   :-0.01957  
 3rd Qu.: 0.121000   3rd Qu.: 1.0090000   3rd Qu.: 0.08450  
 Max.   : 0.997000   Max.   : 1.8970000   Max.   : 0.99500  
       f6                  f7                  f8                 f9           
 Min.   :-0.999000   Min.   :-1.940000   Min.   :-1.86300  

### Split into Test vs. Train

Check frequency distribution of the target vaariable.

In [5]:
nrow(mainDF[mainDF$TARGET == -1,])
nrow(mainDF[mainDF$TARGET == 1,])/nrow(mainDF)

[1] 417

[1] 0.5070922

About a 50/50 split. Partition into test v. training sets.

In [6]:
num_obs = nrow(mainDF)

set.seed(865309) 

index_shuffled <- sample(num_obs)
training_ratio <- 0.66
training_size <- round(num_obs * training_ratio)
training_index <- index_shuffled[1:training_size]
testing_index <- index_shuffled[(training_size+1):num_obs]
trainset <- mainDF[training_index,]
testset <- mainDF[testing_index,]

In [7]:
nrow(mainDF[mainDF$TARGET == 1,])/nrow(mainDF)
nrow(trainset[trainset$TARGET == 1,])/nrow(trainset)
nrow(testset[testset$TARGET == 1,])/nrow(testset)

[1] 0.5070922

[1] 0.5071685

[1] 0.5069444

Got that 50/50 split for test & train; looking good.

## Using a Two-Class Support Vector Machine

In [8]:
library(e1071)
library(ROCR)

Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



In [9]:
head(testset,3)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,⋯,f27,f28,f29,f30,f31,f32,f33,f34,f35,TARGET
774,-0.023,-0.811,-0.464,-0.436,-0.766,-0.677,-0.091,-0.895,-0.342,0.110,⋯,0.375,-0.077,-0.001,0.068,-0.403,-0.638,-0.690,-1.026,0.843,-1
694,0.414,-0.447,0.184,-0.432,-0.449,0.209,0.000,-1.074,-0.849,0.011,⋯,0.020,-1.297,0.118,0.107,0.381,-1.125,0.500,-0.488,0.154,-1
13,0.655,-0.690,0.184,0.472,-0.513,-0.090,-0.646,0.062,0.799,-0.111,⋯,-0.100,-0.280,-0.063,0.057,-0.179,0.011,0.705,-0.085,-0.813,-1


In [10]:
head(testset[-37],3)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,⋯,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35
774,-0.023,-0.811,-0.464,-0.436,-0.766,-0.677,-0.091,-0.895,-0.342,0.110,⋯,0.740,0.375,-0.077,-0.001,0.068,-0.403,-0.638,-0.690,-1.026,0.843
694,0.414,-0.447,0.184,-0.432,-0.449,0.209,0.000,-1.074,-0.849,0.011,⋯,0.606,0.020,-1.297,0.118,0.107,0.381,-1.125,0.500,-0.488,0.154
13,0.655,-0.690,0.184,0.472,-0.513,-0.090,-0.646,0.062,0.799,-0.111,⋯,-0.195,-0.100,-0.280,-0.063,0.057,-0.179,0.011,0.705,-0.085,-0.813


## First SVM: Linear

In [11]:
# Linear kernel
svm.model <- svm(TARGET ~ ., data = trainset, cost = 1, kernel='linear') 
svm.pred <- predict(svm.model, testset[,-37])

pred <- prediction(svm.pred, testset$TARGET)
auc <- performance(pred, "auc")
print(paste("AUC=", auc@y.values[[1]], sep=""))

[1] "AUC=0.968743970673355"


A simple linear model yields AUC of 0.9687.  Not too bad.  Time to start keeping score.

|SVM Kernel|Tuning|AUC              |
|----------|------|-----------------|
|linear    |cost=1|0.968743970673355|

And create a function so we can test more quickly.

In [41]:
test_svm_all_defaults <- function(trainDF, testDF, theKernel,  theDegree = 3, 
                                  theGamma = -1, theCoef0 = 0, theCost = 1,
                                  theEpsilon = 0.1, printSummary = FALSE) {
    if (theGamma == -1){    
        if (theKernel == 'RBF'){
           svm.model <- svm(TARGET ~ ., data = trainDF, 
                            degree = theDegree, coef0 = theCoef0,cost = theCost, 
                            epsilon = theEpsilon)          
        } else {
           svm.model <- svm(TARGET ~ ., data = trainDF, kernel=theKernel, 
                            degree = theDegree, coef0 = theCoef0,cost = theCost,
                            epsilon = theEpsilon)         
        }
    } else {
        if (theKernel == 'RBF'){
           svm.model <- svm(TARGET ~ ., data = trainDF, 
                            degree = theDegree, coef0 = theCoef0,cost = theCost, 
                            epsilon = theEpsilon, gamma = theGamma)          
        } else {
           svm.model <- svm(TARGET ~ ., data = trainDF, kernel=theKernel, 
                            degree = theDegree, coef0 = theCoef0,cost = theCost,
                            epsilon = theEpsilon, gamma = theGamma)         
        }        
    }


    if (printSummary){
        print(summary (svm.model))
    }
    
    svm.pred <- predict(svm.model, testDF[,-37])

    pred <- prediction(svm.pred, testDF$TARGET)
    auc <- performance(pred, "auc")
    print(paste("AUC=", auc@y.values[[1]], sep=""))
}

Now try a bunch of svm models, all with defaults

In [42]:
test_svm_all_defaults(trainset, testset, 'linear')
test_svm_all_defaults(trainset, testset, 'polynomial')
test_svm_all_defaults(trainset, testset, 'RBF')
test_svm_all_defaults(trainset, testset, 'sigmoid')

[1] "AUC=0.968743970673355"
[1] "AUC=0.885828670654063"
[1] "AUC=0.977570904881343"
[1] "AUC=0.809039166505885"


|SVM Kernel|AUC              |
|----------|-----------------|
|linear    |0.968743970673355|
|polynomial|0.885828670654063|
|RBF       |0.977570904881343|
|sigmoid   |0.809039166505885|

Okay, so RBF the default kernel, with default parameters, yeilds 0.9776 AUC.

#### Try some parameters

In [39]:
test_svm_all_defaults(trainset, testset, 'linear', printSummary=TRUE)


Call:
svm(formula = TARGET ~ ., data = trainDF, kernel = theKernel, degree = theDegree, 
    coef0 = theCoef0, cost = theCost)


Parameters:
   SVM-Type:  eps-regression 
 SVM-Kernel:  linear 
       cost:  1 
      gamma:  0.02777778 
    epsilon:  0.1 


Number of Support Vectors:  471





[1] "AUC=0.968743970673355"


In [40]:
tune.out<-tune(svm ,TARGET ~ ., data = trainset,  ,kernel ="linear",
        ranges =list(cost=c(0.001 , 0.01, 0.1, 1,5,10,100) ))
summary(tune.out)

# get the best model
bestmod <-tune.out$best.model
summary (bestmod )


Parameter tuning of ‘svm’:

- sampling method: 10-fold cross validation 

- best parameters:
 cost
  100

- best performance: 0.3881963 

- Detailed performance results:
   cost     error dispersion
1 1e-03 0.8568592 0.14758215
2 1e-02 0.6229603 0.07652119
3 1e-01 0.4392814 0.18625185
4 1e+00 0.4004802 0.21269152
5 5e+00 0.3920089 0.19928967
6 1e+01 0.3887190 0.19181545
7 1e+02 0.3881963 0.18289181



Call:
best.tune(method = svm, train.x = TARGET ~ ., data = trainset, ranges = list(cost = c(0.001, 
    0.01, 0.1, 1, 5, 10, 100)), kernel = "linear")


Parameters:
   SVM-Type:  eps-regression 
 SVM-Kernel:  linear 
       cost:  100 
      gamma:  0.02777778 
    epsilon:  0.1 


Number of Support Vectors:  481






In [43]:
test_svm_all_defaults(trainset, testset, 'linear')
test_svm_all_defaults(trainset, testset, 'linear', theGamma = 0.02777778, theCost = 100, theEpsilon = 0.1)

[1] "AUC=0.968743970673355"
[1] "AUC=0.9704321821339"


In [ ]:
test_svm_all_defaults(trainset, testset, 'linear')
test_svm_all_defaults(trainset, testset, 'linear')
test_svm_all_defaults(trainset, testset, 'linear')
test_svm_all_defaults(trainset, testset, 'linear')
test_svm_all_defaults(trainset, testset, 'linear')
test_svm_all_defaults(trainset, testset, 'linear')
test_svm_all_defaults(trainset, testset, 'linear')


#### Play with Polynomial parameters.

In [22]:
test_svm_all_defaults(trainset, testset, 'polynomial', theDegree = 0)
test_svm_all_defaults(trainset, testset, 'polynomial', theDegree = 1)
test_svm_all_defaults(trainset, testset, 'polynomial', theDegree = 2)
test_svm_all_defaults(trainset, testset, 'polynomial', theDegree = 3)
test_svm_all_defaults(trainset, testset, 'polynomial', theDegree = 4)
test_svm_all_defaults(trainset, testset, 'polynomial', theDegree = 5)

[1] "AUC=0.5"
[1] "AUC=0.925670461122902"
[1] "AUC=0.854427937487942"
[1] "AUC=0.885828670654063"
[1] "AUC=0.842272814972025"
[1] "AUC=0.811595601003279"


No big surpise that theDegree of 0 yiels 0.5; and 1 gives the same results as linear.  What's interesting, is that as you increase the degre, AUC degrades.'

In [24]:
test_svm_all_defaults(trainset, testset, 'polynomial', theCoef0 = 1, theDegree = 3)
test_svm_all_defaults(trainset, testset, 'polynomial', theCoef0 = 2, theDegree = 3)
test_svm_all_defaults(trainset, testset, 'polynomial', theCoef0 = 3, theDegree = 3)
test_svm_all_defaults(trainset, testset, 'polynomial', theCoef0 = 4, theDegree = 3)
test_svm_all_defaults(trainset, testset, 'polynomial', theCoef0 = 5,theDegree = 3)

[1] "AUC=0.983600231526144"
[1] "AUC=0.986204900636697"
[1] "AUC=0.98519197376037"
[1] "AUC=0.983214354620876"
[1] "AUC=0.981912020065599"


In [ ]:
test_svm_all_defaults(trainset, testset, 'RBF')


In [26]:
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 1)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 2)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 3)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 4)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 5)

[1] "AUC=0.809039166505885"
[1] "AUC=0.642581516496238"
[1] "AUC=0.5990256608142"
[1] "AUC=0.7433918579973"
[1] "AUC=0.744645957939418"
[1] "AUC=0.744645957939418"


In [27]:
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.1)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.2)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.3)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.4)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.5)

[1] "AUC=0.809039166505885"
[1] "AUC=0.75694578429481"
[1] "AUC=0.650829635346324"
[1] "AUC=0.664624734709628"
[1] "AUC=0.657486011962185"
[1] "AUC=0.640893305035694"


In [28]:
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.1)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.01)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.001)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.0001)
test_svm_all_defaults(trainset, testset, 'sigmoid', theCoef0 = 0.00001)

[1] "AUC=0.809039166505885"
[1] "AUC=0.75694578429481"
[1] "AUC=0.809714451090103"
[1] "AUC=0.80918387034536"
[1] "AUC=0.814779085471735"
[1] "AUC=0.750868223036852"


In [15]:
### Try 10-fold cross validation

## Try an SVM with Sigmoid Kernel

In [16]:
gamma = .1
kernel = 'sigmoid'
svm.model <- svm(TARGET ~ ., data = trainset, cost = 1, gamma = gamma)#, kernel=kernel) 
svm.pred <- predict(svm.model, testset[,-37])

pred <- prediction(svm.pred, testset$TARGET)
auc <- performance(pred, "auc")
print(paste("AUC=", auc@y.values[[1]], sep=""))

[1] "AUC=0.933822110746672"


|SVM Kernel|Tuning                                            |AUC              |
|----------|--------------------------------------------------|-----------------|
|linear    |cost=1 (defualt)                                  |0.968743970673355|
|sigmoid   |cost=1 (defualt); gamma = 0.1                     |0.933822110746672|

## Back to the Question(s) at Hand
**Use the veh-prime dataset to classify cars vs. non-cars with SVMs.  Try 10 fold cross-validation, tune parameters and observe results.**

## Using package e1071

In [17]:
#install the package
install.packages("e1071")

Installing package into ‘/home/vagrant/R/x86_64-pc-linux-gnu-library/3.2’
(as ‘lib’ is unspecified)



The downloaded source packages are in
	‘/tmp/Rtmp566l6i/downloaded_packages’


In [18]:
# load the library
# Note that e1071 contains other classification and clustering methods and is a useful library to keep in your toolbox.
library(e1071)

In [19]:
# svm i sused to train a support vector machine
# Various arguments are interesting - In this case you will see that we are using a linear kernel
# You can also explore other kerney and observe how the decision boundary changes.
# Various kernels are available - linear, polynomial, radial basis, and sigmoid.


# You will also notice that we are using a cost = 10. 
# Cost refers to the C constant of the regularization term in a Lagrang formulae
svmfit<-svm(TARGET~., data=autosdf, kernel="linear", cost=10, scale=FALSE)

ERROR: Error in eval(expr, envir, enclos): object 'autosdf' not found


In [ ]:
# Visualize the decision boundary
# plot(svmfit , autosdf)
svmfit$index
summary (svmfit )

In [ ]:
# Try using a smaller cost value and see how it impacts the decision boundary
svmfit<-svm(TARGET~., data=autosdf, kernel="linear", cost=0.1, scale=FALSE)

In [ ]:
# plot(svmfit , autosdf)
svmfit$index
summary (svmfit )

## Using a Two-Class LocallyDeep Support Vector Machine (MSR)